# Kmeans Clustering

**KMeans runs quickly (O(n)) but does not handle elongated clusters or non linear data well AND you have to specify the number of clusters up front (how would you possibly know this?). AND it is vulnerable to outliers because it will insist on placing them in a cluster** <br>
The following illustrates a sample kmeans implementation to show the algorithm in action. For practical problems, use the kmeans implementation in scikitlearn (see the bottom of this notebook for sample code)

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
# plt.style.use('default')
import pandas as pd
import numpy as np
#want to filter the seaborn warnings
import warnings
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")
warnings.filterwarnings("ignore", category=FutureWarning)

## Constants and Functions

In [2]:
NUMB_SAMPLES=500
K_CLUSTERS = 3
CLUSTER_STD = 1.0  #increase from 1 to 3 to make clusters overlap

# RANDOM_STATE=7  #7 generates bad clusters because the initial points cluster centers were chosen poorly 
RANDOM_STATE=999  # this will choose better for this problem (kmeans++ does a good job of solving this problem, see below)
UNKNOWN =-1

import random
random.seed(RANDOM_STATE)
def get_random_centroids(X,k):
    '''
    chooses k centroids randomly from
    the number of points in X
    '''
    samps=X.sample(n=k, random_state=RANDOM_STATE) #choose 3 random points as initial centroids
    samps.reset_index(drop=True, inplace=True)  #replace orig index numbers with sequential ones
    samps.drop(columns=['old_cluster_guess'], inplace=True) 
    samps.rename(columns={'cluster_guess':'cluster_number'}, inplace=True)
    samps['cluster_number']=samps.index #assign cluster number to each centroid
    return samps

from scipy.spatial.distance import pdist, squareform
def get_furthest_centroids(X,k):
    '''
    chooses k centroids by choosing the furthest
    points from each other in X
    '''
    # Calculate the pairwise distances between all points
    distances = pdist(X[['X0', 'X1']], metric='euclidean')
    distance_matrix = squareform(distances)

    # Find the indices of the 3 points that are furthest away from each other
    furthest_points_indices = np.unravel_index(np.argsort(distance_matrix, axis=None)[-3:], distance_matrix.shape)

    # Get the unique indices of the furthest points
    unique_indices = np.unique(furthest_points_indices)
    
    # Get the furthest points
    furthest_points = X.iloc[unique_indices].copy()
    furthest_points.reset_index(drop=True, inplace=True)
    furthest_points.drop(columns=['old_cluster_guess'], inplace=True) 
    furthest_points.rename(columns={'cluster_guess':'cluster_number'}, inplace=True)
    furthest_points['cluster_number']=furthest_points.index #assign cluster number to each centroid
    return furthest_points


def plot_clusters(X, centroids=None, figsize=(5,3)):
    '''
    plots the data we are trying to cluster and the centroids
    X: points to cluster (x,y)
    centroids: centers of each cluster
    '''
    fig = plt.figure(figsize=figsize);
    #notice that I'm plotting up to 3 scatterplots on the same figure
    changedX=X[X.old_cluster_guess!=X.cluster_guess]  #how many changed since last time
    if(len(changedX)>0):
        sns.scatterplot(data=X[X.old_cluster_guess!=X.cluster_guess],x="X0",y="X1",hue='old_cluster_guess',s=40, legend=False, palette='Accent');
    sns.scatterplot(data=X,x="X0",y="X1",hue='cluster_guess',s=10, legend=False, palette='Accent');
    if(centroids is not None):
        sns.scatterplot(data=centroids,x="X0",y="X1",s =200,hue='cluster_number', palette='Accent',legend=False);

from sklearn.cluster import kmeans_plusplus
from sklearn.datasets import make_blobs
def generate_sample_data():
    #Generate some clusters
    #note that y denotes group membership, something we are trying to predict
    X,y=make_blobs(n_samples=NUMB_SAMPLES,
                n_features=2,
                centers=K_CLUSTERS,
                cluster_std=CLUSTER_STD,  #1 gives distinct clusters, 4 will give overlapping clusters
                shuffle=True,
                random_state=RANDOM_STATE)

    #place in DataFrame
    X=pd.DataFrame(data=X, columns=["X0","X1"])
    X['cluster_guess']=UNKNOWN
    X['old_cluster_guess']=UNKNOWN
    return X
import math
def find_closest_cluster(ser, centroids):
    # which cluster is this series point closest to
    dsts = []  
    for _,c in centroids.iterrows():
        dsts.append(math.dist( (ser[0],ser[1]),(c[0],c[1])))

    #which centroid
    return (dsts.index(min(dsts)))

def find_mean(df):
    #find the mean
    return df.mean()

## Generate some data 
generate a synthetic dataset using sklearns makeblobs , see <a href="https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html">Make Blobs</a>

In [ ]:
X=generate_sample_data()

#want to see them?
plot_clusters(X)

# K means, General Algorithm
<ol>
    <li>Randomly pick *k* centroids from sample points as initial cluster centers</li>
    <li>Assign each sample to the nearest centroid</li>
    <li>Move the centroids to the center of the samples assigned to it</li>
    <li>Repeat steps 2 and 3 until cluster membership stops changing</li>
</ol>

## Pick *k* centroids from sample points as initial cluster centers

Either randomly, or the kmeans++ way

In [ ]:
#random 
# centroids=get_random_centroids(X,K_CLUSTERS)

#kmeans++ ish (find points furthest away from each other)
centroids=get_furthest_centroids(X,K_CLUSTERS)
plot_clusters(X, centroids)
# centroids

## Show Kmeans converging - Repeat this cell until no points change clusters

Assign each sample to the nearest centroid then move the centroids to the mean of their assigned clusters
<mark>NOTE Points that changed will have their old centroid color as an outline and their new centroid color in the center

In [ ]:
#assign points to a cluster
X['old_cluster_guess']=X['cluster_guess']
X['cluster_guess']=X.apply(find_closest_cluster, centroids=centroids, axis=1)  #assign points to closest cluster

new_centroid_center = X.groupby('cluster_guess').apply(find_mean)   #get new cluster mean
centroids['X0']=new_centroid_center['X0']
centroids['X1']=new_centroid_center['X1']

numb_points_changed_clusters=sum(X.old_cluster_guess!=X.cluster_guess)  
print(f'{numb_points_changed_clusters} points changed clusters')
plot_clusters(X, centroids)

# sklearns kmeans implementation
Find sample code <a href="https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html">here</a><br>
<mark>Prefer to initialize the kmeans algorithm with k-means++, this puts the initial cluster center choices as far away from each other as possible which increases the chances for convergence

## Sample use

In [6]:
#get a copy of original data
X_cpy =generate_sample_data()

#we need a numpy array for sklearns k_means
Xnp=X_cpy.loc[:,['X0','X1']].to_numpy()

In [7]:
from sklearn.cluster import KMeans

# initialize with k-means++, this puts the initial cluster choices as far away from each other
# as possible which increases the chances for convergence
kmeans = KMeans(n_clusters=3, init='k-means++', random_state=RANDOM_STATE)
kmeans=kmeans.fit(Xnp)  #iteratively calculates the clusters

#lets see what we have
X_cpy['cluster_guess']=pd.Series(kmeans.labels_)
X_cpy['old_cluster_guess']=pd.Series(kmeans.labels_)

In [ ]:
a=pd.DataFrame(kmeans.cluster_centers_, columns=['X0','X1'])
a['cluster_number']=a.index

plot_clusters(X_cpy, a)

In [ ]:
#once fitted you can predict what clusters new data would belong too
kmeans.predict([[0,0],[-7.5,3]])

In [ ]:
#and see what clusters all the fitted data belong to
kmeans.labels_

In [ ]:
#where are the cluster centers?
kmeans.cluster_centers_

In [ ]:
#convert cluster centers to dataframe to use the above plotting function
centroids1=pd.DataFrame(data=kmeans.cluster_centers_, columns=['X0', 'X1'])
centroids1.reset_index( inplace=True)
centroids1.rename(columns={'index':'cluster_number'}, inplace=True)
centroids1

In [ ]:
plot_clusters(X_cpy, centroids1)

## Inertia: For each cluster, square the sum of the distance between every point in that particular cluster and the cluster centroid.  Add these cluster sums together. 
![](./SSEKmeans.png)

What good is it? It helps you calculate the correct number of clusters for one

In [ ]:
#the inertia for the above sklearn implementation
kmeans.inertia_

## The elbow method, used to find optimal number of clusters for kmeans
Calculate the inertia for different number of clusters.  As the number of clusters increase the inertia will decrease ( more clusters means each cluster is smaller so the SSE will also be smaller given that the points will also be closer to cluster centers)<br>
The idea is to identify the value of K where the inertia starts to rapidly increase. This means that the clusters are aquiring member points that are further and further away.

In [15]:
# First  get the inertias for various K's
inertias=[]
for k in range (1,11):
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=RANDOM_STATE).fit(Xnp)
    inertias.append(kmeans.inertia_)


In [ ]:
inertias

In [ ]:
#then plot them
fig, ax = plt.subplots()

plt.plot( range(1,11), inertias, marker='o')
# ax.annotate('looks OK', xy=(3, 800), xytext=(4, 1500),
#             arrowprops=dict(facecolor='black', shrink=0.05))
plt.xlabel=('Number of clusters')
plt.ylabel('Inertia')
plt.show()

Inertia starts to rapidly increase from 3 to 2 clusters.

### Unfortunately the elbow method (and following silhouette plots) are often ambiguous when the clusters are not linearly seperable

<mark>Increase the CLUSTER_STD param in second cell from 1.0 to 3.0 then rerun the elbow method above. make_blobs (in function generate_sample_data) will create clusters that are no longer linearly seperable which makes the elbow method hard to interpret.  

## Silhouette Plots, a way to visually evaluate the quality of clustering (works with other cluster methods as well)
Works with all distance based clustering methods,  see the reference on class website titled 'Selecting the number of clusters with silhouette analysis' for explanation and use

Steps to calculate silhouette score

1. For each data point, the average distance (a_i) to other data points within the same cluster is calculated. This value represents the similarity level of the data point to others in its cluster.

2. For each data point, the average distance (b_i) to all other clusters it doesn’t belong to is computed. This value indicates how different the data point is from data points in other clusters.

3. The Silhouette score is calculated using the formula:

Silhouette Score = (b_i — a_i) / max(a_i, b_i)

4. By taking the average of the Silhouette scores calculated for each data point, an overall Silhouette score is obtained, which measures the success of clustering results.

Key characteristics of the Silhouette score include:<mark>
It ranges from -1 to +1:</mark>

Positive values indicate that data points belong to the correct clusters, indicating good clustering results.

A score of zero suggests overlapping clusters or data points equally close to multiple clusters.

Negative values indicate that data points are assigned to incorrect clusters, indicating poor clustering results.

A higher Silhouette score indicates better clustering results.


In [18]:
from sklearn.metrics import silhouette_samples, silhouette_score

In [ ]:
# get average silhoutte for various K's (calculated by averaging each points score)
silhouette_avgs=[]
for k in range(2,11):
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=RANDOM_STATE).fit(Xnp)
    print(f"for {k} clusters, silhouette_score={silhouette_score(Xnp, list(kmeans.labels_))}")
    # silhouette_avgs.append(silhouette_score(Xnp, kmeans.labels_))